In [1]:
import findspark
findspark.init("/usr/local/spark")
import pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python Spark SQL example").getOrCreate()

In [4]:
salestxnDF = spark.read.load("salestxn_header.txt", format="csv", sep="\t", inferSchema="true", header="true")

In [5]:
salestxnDF.printSchema()

root
 |-- txn_id: integer (nullable = true)
 |-- txn_date: string (nullable = true)
 |-- txn_cust_id: integer (nullable = true)
 |-- txn_amount: integer (nullable = true)
 |-- txn_item_category: string (nullable = true)
 |-- txn_item_name: string (nullable = true)
 |-- txn_city: string (nullable = true)
 |-- txn_state: string (nullable = true)
 |-- txn_payment: string (nullable = true)



In [6]:
salestxnDF.createOrReplaceTempView("salestxn")

In [8]:
stxnDF = spark.sql("SELECT * from salestxn").show()

+------+----------+-----------+----------+--------------------+--------------------+----------+-------------+-----------+
|txn_id|  txn_date|txn_cust_id|txn_amount|   txn_item_category|       txn_item_name|  txn_city|    txn_state|txn_payment|
+------+----------+-----------+----------+--------------------+--------------------+----------+-------------+-----------+
|   103|06-01-2014|    5000002|       558|  Exercise & Fitness|Weightlifting Mac...|  Gulbarga|    Karnataka|     credit|
|   124|05-02-2014|    5000009|      9950|          Gymnastics|    Gymnastics Rings|   Gurgaon|      Haryana|     credit|
|   130|06-03-2014|    5000001|     12690|  Outdoor Recreation|             Hunting|   Aligarh|Uttar Pradesh|     credit|
|   155|10-03-2014|    5000007|     15487|  Outdoor Recreation|             Running| Bangalore|    Karnataka|     credit|
|   153|02-04-2014|    5000005|      4482|Outdoor Play Equi...|   Lawn Water Slides|   Asansol|       Bengal|       cash|
|   104|06-05-2014|    5

In [9]:
stxnDF = spark.sql("SELECT * from salestxn WHERE txn_amount > 10000").show()

+------+----------+-----------+----------+--------------------+--------------------+-------------+--------------+-----------+
|txn_id|  txn_date|txn_cust_id|txn_amount|   txn_item_category|       txn_item_name|     txn_city|     txn_state|txn_payment|
+------+----------+-----------+----------+--------------------+--------------------+-------------+--------------+-----------+
|   130|06-03-2014|    5000001|     12690|  Outdoor Recreation|             Hunting|      Aligarh| Uttar Pradesh|     credit|
|   155|10-03-2014|    5000007|     15487|  Outdoor Recreation|             Running|    Bangalore|     Karnataka|     credit|
|   104|06-05-2014|    5000003|     19819|          Gymnastics|    Gymnastics Rings|      Chennai|    Tamil Nadu|     credit|
|   139|08-05-2014|    5000007|     15060|  Outdoor Recreation|Camping & Backpac...|      Kolkata|        Bengal|     credit|
|   141|11-07-2014|    5000005|     16510|         Team Sports|        Cheerleading|   Coimbatore|    Tamil Nadu|     

In [11]:
newformatDF = spark.sql("SELECT txn_id, txn_date, txn_cust_id, txn_amount, (txn_item_category, txn_item_name) as txn_item_details, txn_city, txn_state, txn_payment FROM salestxn WHERE txn_amount > 10000")

In [14]:
newformatDF.show()

+------+----------+-----------+----------+--------------------+-------------+--------------+-----------+
|txn_id|  txn_date|txn_cust_id|txn_amount|    txn_item_details|     txn_city|     txn_state|txn_payment|
+------+----------+-----------+----------+--------------------+-------------+--------------+-----------+
|   130|06-03-2014|    5000001|     12690|[Outdoor Recreati...|      Aligarh| Uttar Pradesh|     credit|
|   155|10-03-2014|    5000007|     15487|[Outdoor Recreati...|    Bangalore|     Karnataka|     credit|
|   104|06-05-2014|    5000003|     19819|[Gymnastics,Gymna...|      Chennai|    Tamil Nadu|     credit|
|   139|08-05-2014|    5000007|     15060|[Outdoor Recreati...|      Kolkata|        Bengal|     credit|
|   141|11-07-2014|    5000005|     16510|[Team Sports,Chee...|   Coimbatore|    Tamil Nadu|     credit|
|   125|06-10-2014|    5000003|     15120|[Water Sports,Sur...|      Nanital|   Uttarakhand|     credit|
|   145|09-11-2014|    5000001|     13537|[Water Sports

In [15]:
newformatDF.coalesce(1).write.json("newformatJSON")